In [2]:
%load_ext autoreload
%autoreload 2
import torch
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os
from typing import List, Tuple, Dict, Union, Any, Optional
from tqdm import tqdm
try:
    os.chdir(os.path.join(os.getcwd(), '../../Language-Technology-Project'))
    print(os.getcwd())
except:
    print("ALready in current dir")

c:\Users\mo\Documents\Organisations\RUG\Language-Technology-Project


# Generative pipeline

In [6]:
# from transformers import GPT2Tokenizer, GPT2LMHeadModel

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# tokenizer = GPT2Tokenizer.from_pretrained('gpt2-xl')
# model = GPT2LMHeadModel.from_pretrained('gpt2-xl').to(device)

from transformers import AutoTokenizer, AutoModelForCausalLM
model = AutoModelForCausalLM.from_pretrained('mosaicml/mpt-1b-redpajama-200b', trust_remote_code=True).to(device)
tokenizer = AutoTokenizer.from_pretrained("EleutherAI/gpt-neox-20b")

print(f"size_gb(model) = {model.num_parameters() * 4 / 1024**3:.2f} GB")
print("vocab size:", tokenizer.vocab_size)

Explicitly passing a `revision` is encouraged when loading a configuration with custom code to ensure no malicious code has been contributed in a newer revision.
Explicitly passing a `revision` is encouraged when loading a model with custom code to ensure no malicious code has been contributed in a newer revision.
C:\Users\mo/.cache\huggingface\modules\transformers_modules\mosaicml\mpt-1b-redpajama-200b\a0929818043de593a6b4248481e18a4b276f5e1e\attention.py:289: UserWarning: Using `attn_impl: torch`. If your model does not use `alibi` or `prefix_lm` we recommend using `attn_impl: flash` otherwise we recommend using `attn_impl: triton`.
  warnings.warn(


You are using config.init_device='cpu', but you can also use config.init_device="meta" with Composer + FSDP for fast initialization.
size_gb(model) = 4.89 GB
vocab size: 50254


In [14]:
def predict_next_token(text):
    encoded_input = tokenizer(text, return_tensors='pt').to(device)
    encoded_input['attention_mask'] = encoded_input['attention_mask'].bool()
    output = model(**encoded_input)
    next_token_logits = output.logits[:, -1, :]
    likelihoods = torch.softmax(next_token_logits, dim=-1)
    sorted_likelihoods, sorted_indices = torch.sort(likelihoods, descending=True)
    return sorted_likelihoods, sorted_indices

text = "Man is a doctor as a women is a"
sorted_likelihoods, sorted_indices = predict_next_token(text)
print("Top 5 tokens:")
for i in range(5):
    print(f"{i+1}: {tokenizer.decode(sorted_indices[0, i])} ({sorted_likelihoods[0, i]:.2f})")

Top 5 tokens:
1:  doctor (0.29)
2:  nurse (0.10)
3:  woman (0.05)
4:  mother (0.03)
5:  house (0.02)


In [15]:
import torch

def get_likelihoods_of_words_given_context(prompt: str, words: list, model, tokenizer):
    """
    Returns a list of likelihoods of words given a prompt.
    """
    likelihoods, _ = predict_next_token(prompt)
    word_probs = []
    for word in words:
        tokens = tokenizer.encode(word)
        # word_probs.append(likelihoods[0, tokenizer.encode(word)[0]].item())
        probability = 1
        for token in tokens:
            probability *= likelihoods[0, token].item()
        word_probs.append(probability)
    
    output = list(zip(words, word_probs))
    output.sort(key=lambda x: x[1], reverse=True)
    return output
# example
prompt = "2+2="
options = ["4", "5", "6", "7", "8", "9"]
get_likelihoods_of_words_given_context(prompt, options, model, tokenizer)


[('4', 0.004680944606661797),
 ('5', 0.0044109150767326355),
 ('6', 0.004199830815196037),
 ('7', 0.004028704948723316),
 ('8', 0.0039849067106842995),
 ('9', 0.003596288850530982)]

## Category classifier

In [ ]:
from settings import categories

class LikelihoodBasedCategoryClassifier:

    def __init__(self, categories):
        self.categories = categories

    def __shuffle_categories(self):
        np.random.shuffle(self.categories)
    
    def __construct_classification_prompt(self, sentence, max_length=128, categories=None):
        prompt = "Categories of Human Value Detection 2023 by name and description: \n\n"
        expectation2category = {}
        for i, category in enumerate(categories):
            name = category["name"]
            description = category["description"]
            examples = category["examples"]
            example = examples[np.random.randint(len(examples))]
            example = example[:max_length] + "..." if len(example) > max_length else example
            # otpions a-z
            prompt += f"- {name}: {description} (e.g., {example})\n"
            expectation2category[name] = name
        prompt += "\n"
        prompt += "Classify the following sentence into one of the categories above:\n"
        prompt += f"\n'Argument: {sentence}"
        prompt += "\Category: "

        return prompt, expectation2category

    def classify_once(self, sentence: str) -> List[float]:
        self.__shuffle_categories()
        prompt, expectations = self.__construct_classification_prompt(sentence, categories=self.categories)
        likelihoods = get_likelihoods_of_words_given_context(prompt, expectations.keys(), model, tokenizer)
        likelihoods = [(expectations[c], p) for c, p in likelihoods]
        return likelihoods
    
    def __call__(self, sentence: str, trial_count=5) -> List[float]:
        results = {category["name"]: 0.0 for category in self.categories}
        for _ in range(trial_count):
            likelihoods = self.classify_once(sentence)
            for category, likelihood in likelihoods:
                results[category] += likelihood / trial_count
        results = list(results.items())
        results.sort(key=lambda x: x[1], reverse=True)
        return results
    

In [ ]:
classifier = LikelihoodBasedCategoryClassifier(categories)
# take a random category 
category = categories[np.random.randint(len(categories))]
sentence = np.random.choice(category["examples"])
print(f"Category: {category['name']}")
print(f"Sentence: {sentence}")

likelihoods = classifier(sentence)
print(likelihoods)
if likelihoods[0][0] == category["name"]:
    print("Correct!")
else:
    print("Incorrect!")

Category: universalism
Sentence: It is good to accept and try to understand those who are different from oneself.
[('power', 1.27265606124638e-06), ('security', 8.745852113634099e-08), ('hedonism', 3.888645540099349e-12), ('universalism', 1.0556086657274182e-12), ('stimulation', 2.221720261672261e-13), ('self-direction', 1.0781985909243936e-16), ('conformity', 2.817910554125732e-17), ('achievement', 1.8855096633304625e-17), ('tradition', 6.323611187755675e-19), ('benevolence', 4.239685788541454e-25)]
Incorrect!


In [ ]:
# basic evaluation go throguuh all categories and all examples
def evaluate_classifier(classifier, categories, trial_count=5):
    results = {category["name"]: [] for category in categories}
    
    for category in categories:
        for sentence in category["examples"]:
            likelihoods = classifier(sentence, trial_count=trial_count)
            results[category["name"]].append(likelihoods)
    # compute accuracy
    correct = 0
    total = 0
    for category in categories:
        for likelihoods in results[category["name"]]:
            if likelihoods[0][0] == category["name"]:
                correct += 1
            total += 1
    accuracy = correct / total
    return accuracy, results

accuracy, results = evaluate_classifier(classifier, categories)
print(f"Accuracy: {accuracy:.2f}")

Accuracy: 0.08
